<a href="https://colab.research.google.com/github/flaghunter21/DFS/blob/main/Contest_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# READ IN DATA

In [618]:
import re

In [619]:
import pandas as pd
contest = pd.read_excel('colab_contest_analyzer.xlsx', sheet_name='standings')
contest.dropna()
contest.head()

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup
0,1,3257533438,ihbrown3 (22/150),0,213.05,P Jordan Montgomery P Kyle Freeland OF Aaron J...
1,2,3257373692,BRORANNOSAURUS_FLEX (77/150),0,209.20,P Michael Lorenzen P Jordan Montgomery OF Shoh...
2,3,3257533499,ihbrown3 (83/150),0,198.05,P Jordan Montgomery P Kyle Freeland OF Aaron J...
3,4,3257484992,pianoclub (30/70),0,197.15,P Jordan Montgomery OF Aaron Judge OF Mike Tro...
4,5,3257062564,giantsquid (38/150),0,194.20,P Michael Lorenzen P Jordan Montgomery OF Shoh...


In [620]:
players = pd.read_excel('colab_contest_analyzer.xlsx', sheet_name='players')
players.head()
players['Player'] = players['Player'].str.replace('-',' ', regex=True) \
  .str.replace('Peter Alonso', 'Pete Alonso') \
  .str.replace('Ha seong Kim', 'Ha Seong Kim')

# CLEAN CONTEST

In [621]:
# Clean Lineup
rep = {'P ':',', ' P ':',', ' C ':',', ' 1B ':',', ' 2B ':',', ' 3B ':',', ' SS ':',', ' OF ':','}
contest['Lineup'] = contest['Lineup'].str.replace(' P ',',') \
  .str.replace('P ','') \
  .str.replace(' C ',',') \
  .str.replace(' 1B ',',') \
  .str.replace(' 2B ',',') \
  .str.replace(' 3B ',',') \
  .str.replace(' SS ',',') \
  .str.replace(' OF ',',') \
  .str.replace('OF ','') \
  .str.replace('-',' ')

In [622]:
contest.head()

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup
0,1,3257533438,ihbrown3 (22/150),0,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh..."
1,2,3257373692,BRORANNOSAURUS_FLEX (77/150),0,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta..."
2,3,3257533499,ihbrown3 (83/150),0,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh..."
3,4,3257484992,pianoclub (30/70),0,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb..."
4,5,3257062564,giantsquid (38/150),0,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta..."


In [623]:
cols = ['FLEX1','FLEX2','FLEX3','FLEX4','FLEX5','FLEX6','FLEX7','FLEX8','FLEX9','FLEX10']
# Split cols
contest[cols] = contest['Lineup'].str.split(',', expand=True)
# Replace accents
contest[cols] = contest[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [624]:
contest.head(7)

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,FLEX7,FLEX8,FLEX9,FLEX10
0,1,3257533438,ihbrown3 (22/150),0,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,Josh Donaldson,Jared Walsh,Max Stassi,Andrew Velazquez
1,2,3257373692,BRORANNOSAURUS_FLEX (77/150),0,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,Jake Cronenworth,Jared Walsh,Brandon Marsh,Andrew Velazquez
2,3,3257533499,ihbrown3 (83/150),0,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,Jared Walsh,Max Stassi,Eduardo Escobar,Luis Rengifo
3,4,3257484992,pianoclub (30/70),0,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,Jose Trevino,Andrew Velazquez,Juan Lagares,Clayton Kershaw
4,5,3257062564,giantsquid (38/150),0,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,Anthony Rendon,Cody Bellinger,Jared Walsh,Andrew Velazquez
5,6,3256952974,bric75 (92/150),0,194.05,"Jordan Montgomery,Kyle Freeland,Shohei Ohtani,...",Jordan Montgomery,Kyle Freeland,Shohei Ohtani,Rafael Devers,Mike Trout,J.D. Martinez,Xander Bogaerts,Brendan Rodgers,Christian Vazquez,Bobby Dalbec
6,7,3256953025,bric75 (143/150),0,192.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,Anthony Rendon,Jared Walsh,Jeff McNeil,Tomas Nido


In [625]:
# Clean EntryName
contest['EntryName'] = contest['EntryName'].apply(lambda x: re.sub('\((.+?)\)','', str(x)))

In [626]:
contest[['EntryName']].head(10)

,EntryName
0,ihbrown3
1,BRORANNOSAURUS_FLEX
2,ihbrown3
3,pianoclub
4,giantsquid
5,bric75
6,bric75
7,giantsquid
8,BRORANNOSAURUS_FLEX
9,KevinRob2128


In [627]:
contest.head(6)

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,FLEX7,FLEX8,FLEX9,FLEX10
0,1,3257533438,ihbrown3,0,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,Josh Donaldson,Jared Walsh,Max Stassi,Andrew Velazquez
1,2,3257373692,BRORANNOSAURUS_FLEX,0,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,Jake Cronenworth,Jared Walsh,Brandon Marsh,Andrew Velazquez
2,3,3257533499,ihbrown3,0,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,Jared Walsh,Max Stassi,Eduardo Escobar,Luis Rengifo
3,4,3257484992,pianoclub,0,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,Jose Trevino,Andrew Velazquez,Juan Lagares,Clayton Kershaw
4,5,3257062564,giantsquid,0,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,Anthony Rendon,Cody Bellinger,Jared Walsh,Andrew Velazquez
5,6,3256952974,bric75,0,194.05,"Jordan Montgomery,Kyle Freeland,Shohei Ohtani,...",Jordan Montgomery,Kyle Freeland,Shohei Ohtani,Rafael Devers,Mike Trout,J.D. Martinez,Xander Bogaerts,Brendan Rodgers,Christian Vazquez,Bobby Dalbec


In [628]:
cleaned_contest = contest[['Rank', 'EntryName', 'Points', 'Lineup', 'FLEX1', 'FLEX2', 'FLEX3', 'FLEX4', 'FLEX5', 'FLEX6', 'FLEX7', 'FLEX8', 'FLEX9', 'FLEX10']].dropna()
cleaned_contest.head()

,Rank,EntryName,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,FLEX7,FLEX8,FLEX9,FLEX10
0,1,ihbrown3,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,Josh Donaldson,Jared Walsh,Max Stassi,Andrew Velazquez
1,2,BRORANNOSAURUS_FLEX,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,Jake Cronenworth,Jared Walsh,Brandon Marsh,Andrew Velazquez
2,3,ihbrown3,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,Jared Walsh,Max Stassi,Eduardo Escobar,Luis Rengifo
3,4,pianoclub,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,Jose Trevino,Andrew Velazquez,Juan Lagares,Clayton Kershaw
4,5,giantsquid,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,Anthony Rendon,Cody Bellinger,Jared Walsh,Andrew Velazquez


# CLEAN PLAYERS

In [629]:
players.columns

Index(['Player', 'Inj', 'Likes', 'Pos', 'Salary', 'Order', 'Hand', 'Team',
       'Opp', 'Time', 'vs. Pitcher', 'Win %', 'Total', 'TeamRuns', 'OppRuns',
       'ERA/AVG', 'wOBA', 'ISO', 'Avg innings', 'K/9', 'WHIP', 'Avg', 'STDV',
       'Ceiling', 'Own', 'vGrade', 'pGrade', 'Proj', 'My Proj', 'Exp.', 'Used',
       'Con.', 'Value', 'Own%', 'Actual Score', 'Actual Val'],
      dtype='object')

In [630]:
# Team Map
team_map = dict(zip(players['Player'], players['Team']))
# Score Map
score_map = dict(zip(players['Player'], players['Actual Score']))
# Ownership Map
own_map = dict(zip(players['Player'], players['Own%']))
# Order Map
order_map = dict(zip(players['Player'], players['Order']))

In [631]:
for col in cols:
  cleaned_contest[col + '_Team'] = cleaned_contest[col].map(team_map)
for col in cols:
  cleaned_contest[col + '_Score'] = cleaned_contest[col].map(score_map)
for col in cols:
  cleaned_contest[col + '_Own'] = cleaned_contest[col].map(own_map)
for col in cols:
  cleaned_contest[col + '_Order'] = cleaned_contest[col].map(order_map)

In [632]:
cleaned_contest.head(6)

,Rank,EntryName,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,...,FLEX1_Order,FLEX2_Order,FLEX3_Order,FLEX4_Order,FLEX5_Order,FLEX6_Order,FLEX7_Order,FLEX8_Order,FLEX9_Order,FLEX10_Order
0,1,ihbrown3,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,...,SP,SP,1,3,2,5,2,5,6,9
1,2,BRORANNOSAURUS_FLEX,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,...,SP,SP,3,2,2,5,2,5,1,9
2,3,ihbrown3,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,...,SP,SP,1,3,2,3,5,6,6,8
3,4,pianoclub,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,...,SP,1,2,5,4,5,8,9,7,SP
4,5,giantsquid,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,...,SP,SP,3,2,4,5,4,7,5,9
5,6,bric75,194.05,"Jordan Montgomery,Kyle Freeland,Shohei Ohtani,...",Jordan Montgomery,Kyle Freeland,Shohei Ohtani,Rafael Devers,Mike Trout,J.D. Martinez,...,SP,SP,3,2,2,3,4,3,6,8


In [633]:
cleaned_contest['Dupes'] = cleaned_contest.groupby(cols)['FLEX1'].transform('size')

In [634]:
order_cols = ['FLEX1_Order', 'FLEX2_Order', 'FLEX3_Order', 'FLEX4_Order', 'FLEX5_Order', 'FLEX6_Order', 'FLEX7_Order', 'FLEX8_Order', 'FLEX9_Order', 'FLEX10_Order']
team_cols = ['FLEX1_Team', 'FLEX2_Team', 'FLEX3_Team', 'FLEX4_Team', 'FLEX5_Team', 'FLEX6_Team', 'FLEX7_Team', 'FLEX8_Team', 'FLEX9_Team', 'FLEX10_Team']

In [635]:
cleaned_contest.head()

,Rank,EntryName,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,...,FLEX2_Order,FLEX3_Order,FLEX4_Order,FLEX5_Order,FLEX6_Order,FLEX7_Order,FLEX8_Order,FLEX9_Order,FLEX10_Order,Dupes
0,1,ihbrown3,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,...,SP,1,3,2,5,2,5,6,9,1
1,2,BRORANNOSAURUS_FLEX,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,...,SP,3,2,2,5,2,5,1,9,1
2,3,ihbrown3,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,...,SP,1,3,2,3,5,6,6,8,1
3,4,pianoclub,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,...,1,2,5,4,5,8,9,7,SP,1
4,5,giantsquid,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,...,SP,3,2,4,5,4,7,5,9,1


In [644]:
cleaned_contest.to_csv('cleaned_contest.csv')

In [637]:
players.to_csv('players.csv')

In [638]:
contest.to_csv('contest.csv')

In [639]:
tired.to_csv('order.csv')

In [668]:
for i in range(1,11):
  cleaned_contest.loc[cleaned_contest[f'FLEX{i}_Order'] == 'SP',f'is_hitter_{i}'] = 0
  cleaned_contest.loc[cleaned_contest[f'FLEX{i}_Order'] != 'SP',f'is_hitter_{i}'] = 1
cleaned_contest.head()

,Rank,EntryName,Points,Lineup,FLEX1,FLEX2,FLEX3,FLEX4,FLEX5,FLEX6,...,is_hitter_1,is_hitter_2,is_hitter_3,is_hitter_4,is_hitter_5,is_hitter_6,is_hitter_7,is_hitter_8,is_hitter_9,is_hitter_10
0,1,ihbrown3,213.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Gleyber Torres,...,0,0,1,1,1,1,1,1,1,1
1,2,BRORANNOSAURUS_FLEX,209.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Rafael Devers,Mike Trout,Will Smith,...,0,0,1,1,1,1,1,1,1,1
2,3,ihbrown3,198.05,"Jordan Montgomery,Kyle Freeland,Aaron Judge,Sh...",Jordan Montgomery,Kyle Freeland,Aaron Judge,Shohei Ohtani,Mike Trout,Francisco Lindor,...,0,0,1,1,1,1,1,1,1,1
3,4,pianoclub,197.15,"Jordan Montgomery,Aaron Judge,Mike Trout,Gleyb...",Jordan Montgomery,Aaron Judge,Mike Trout,Gleyber Torres,Anthony Rendon,Jared Walsh,...,0,1,1,1,1,1,1,1,1,0
4,5,giantsquid,194.20,"Michael Lorenzen,Jordan Montgomery,Shohei Ohta...",Michael Lorenzen,Jordan Montgomery,Shohei Ohtani,Mike Trout,Max Muncy,Will Smith,...,0,0,1,1,1,1,1,1,1,1
